## Initializations

In [1]:
# Dependencies
import os
from bs4 import BeautifulSoup as bs, re
from splinter import Browser
import pandas as pd

In [2]:
# Chrome Driver
executable_path = {'executable_path': '/Chromedrv/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Function to return Beautifulsoup

In [3]:
def Create_BeautifulSoup(url,browser):
    browser.visit(url)
    
    # Get html content of the visited page
    html_content = browser.html

    # Create a Beautiful Soup object, for the browser html
    soup = bs(html_content, 'html.parser')
    
    return soup    

In [5]:
# Sample call for function Create_BeautifulSoup
url = 'https://mars.nasa.gov/news'
soup = Create_BeautifulSoup(url,browser)
NewsSlide = soup.find('section',class_='grid_gallery').find('div',class_='list_text')
news_title = NewsSlide.find('div',class_='content_title').text
print(news_title)

NASA's Treasure Map for Water Ice on Mars


## Scrape NASA Mars News

In [6]:
#URL for NASA News Website
url = 'https://mars.nasa.gov/news'

# Create a Beautiful Soup object, for the browser html
soup = Create_BeautifulSoup(url,browser)

In [7]:
#Search for the 1st article/news
NewsSlide = soup.find('section',class_='grid_gallery').find('div',class_='list_text')
# NewsSlide

In [8]:
#Scrate the title and article teaser
news_title = NewsSlide.find('div',class_='content_title').text
news_teaser = NewsSlide.find('div',class_='article_teaser_body').text

In [9]:
print(f"Title: {news_title}")
print(f"Teaser: {news_teaser}")

Title: NASA's Treasure Map for Water Ice on Mars
Teaser: A new study identifies frozen water just below the Martian surface, where astronauts could easily dig it up.


## JPL Mars Space Images - Featured Image

In [10]:
#URL for JPL 
jpl_url = 'https://www.jpl.nasa.gov'

# URL to search for featured Mars image
url = jpl_url + '/spaceimages/index.php?category=Mars'

# Create a Beautiful Soup object, for the browser html
soup = Create_BeautifulSoup(url,browser)

In [11]:
#Search for the 1st article/news
ImageSlide = soup.find('ul',class_='articles').find('a',class_='fancybox')
# ImageSlide

In [12]:
#Scrate the title and article teaser
featured_image_url = jpl_url + ImageSlide['data-fancybox-href']
print(f"Featured Mars Image: {featured_image_url}")

Featured Mars Image: https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA23579_hires.jpg


## Mars Weather

In [13]:
# URL of Mars weather twitter account
url = 'https://twitter.com/marswxreport?lang=en'

# Create a Beautiful Soup object, for the browser html
soup = Create_BeautifulSoup(url,browser)

In [14]:
#Search for the latest tweet
tweetContainer = soup.find('div',class_='js-tweet-text-container').find('p')
tweetImageText = tweetContainer.find('a').text
mars_weather = tweetContainer.text
mars_weather = mars_weather.replace(tweetImageText, "")

print(f"Mars Weather Tweet: {mars_weather}")

Mars Weather Tweet: InSight sol 373 (2019-12-14) low -98.1ºC (-144.6ºF) high -20.1ºC (-4.1ºF)
winds from the SW at 4.9 m/s (11.1 mph) gusting to 19.7 m/s (44.0 mph)
pressure at 6.60 hPa


## Mars Facts

In [15]:
# URL of Mars facts
url = 'https://space-facts.com/mars/'

# Create a Beautiful Soup object, for the browser html
soup = Create_BeautifulSoup(url,browser)

In [16]:
#Search for the 1st article/news
#create Disctionary to store data, 
mars_facts_list = []
mars_fact_dict = dict()
mars_fact_table = soup.find(id="tablepress-p-mars-no-2").find_all("tr")
for mars_fact in mars_fact_table:
    fact_cell = mars_fact.find_all("td")    
    mars_fact_dict["title"] = fact_cell[0].text.replace(":","")
    mars_fact_dict["detail"] = fact_cell[1].text
    mars_facts_list.append(dict(mars_fact_dict))

print(mars_facts_list)

[{'title': 'Equatorial Diameter', 'detail': '6,792 km'}, {'title': 'Polar Diameter', 'detail': '6,752 km'}, {'title': 'Mass', 'detail': '6.39 × 10^23 kg (0.11 Earths)'}, {'title': 'Moons', 'detail': '2 (Phobos & Deimos)'}, {'title': 'Orbit Distance', 'detail': '227,943,824 km (1.38 AU)'}, {'title': 'Orbit Period', 'detail': '687 days (1.9 years)'}, {'title': 'Surface Temperature ', 'detail': '-87 to -5 °C'}, {'title': 'First Record', 'detail': '2nd millennium BC'}, {'title': 'Recorded By', 'detail': 'Egyptian astronomers'}]


## Mars Hemispheres

In [17]:
# URL of Mars facts
base_url = "https://astrogeology.usgs.gov"
url = base_url + '/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# Create a Beautiful Soup object, for the browser html
soup = Create_BeautifulSoup(url,browser)

In [18]:
# get all sections containing the result
hemisphereSlides = soup.find("div",class_="results").find_all("div",class_="description")

In [19]:
# Create variables to store Image Link detail. 
imageDict = {}
hemisphere_image_urls = []

# Loop through each of the result, and get the full resolution image link
for slide in hemisphereSlides:    
    hemisphereName = slide.find('a',class_='product-item').text.replace("Enhanced","").strip()
    hemisphereUrl = base_url + slide.a["href"]      
    
    print(f"processng link: {hemisphereName}") 
    # browse each url, to get the enhanced image link    
    
    # Create a Beautiful Soup object, for the browser html
    soup2 = Create_BeautifulSoup(hemisphereUrl,browser)
    
    #Search the line item, in the download dection
    ImageURL = soup2.find("div",class_="downloads").find("a",text="Sample")
    
    #Create dictionary, of detail captured
    imageDict["title"] = hemisphereName
    imageDict["img_url"] = ImageURL.get("href")
    
    # Append the dictionary to the final list
    hemisphere_image_urls.append(dict(imageDict))

processng link: Cerberus Hemisphere
processng link: Schiaparelli Hemisphere
processng link: Syrtis Major Hemisphere
processng link: Valles Marineris Hemisphere


In [20]:
print("Hemisphere Image List:")
print(hemisphere_image_urls)

Hemisphere Image List:
[{'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [21]:
#Close the browser, once complete.
browser.quit()